The objective of this notebook is to divide the input audios into 5sec time period

In [19]:
import pandas as pd
import numpy as np

import librosa as lb
import soundfile as sf 

import os

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


importing csv detection-info file

In [21]:
detect = pd.DataFrame([[0, "sameera"],[1, "mugil"]], columns=["id", "detection"])
ds = detect['detection'].unique()
ds

array(['sameera', 'mugil'], dtype=object)

filename format has specific meaning, getting them by splitting using '_'

In [ ]:
#getting files name
audio_loc = r'drive/MyDrive/EPITA/Signal Processing/Assi 1/raw_audios'
files = [audio.split('.')[0] for audio in os.listdir(path = audio_loc) if '.wav' in audio]

def tokenize_file(filename):
    return filename.split('_')

In [23]:
files

['ID1_audio2', 'ID1_audio1', 'ID0_audio2', 'ID0_audio1']

In [24]:
files_df = pd.read_csv('drive/MyDrive/EPITA/Signal Processing/Assi 1/my_audio_subslices_5s.csv',names=['start(mins-secs)','end(mins-secs)','start','end','NW/BNC/Intro','filename','id','len'])
files_df = pd.merge(files_df,detect,on='id')
files_df.head()

,start(mins-secs),end(mins-secs),start,end,NW/BNC/Intro,filename,id,len,detection
0,0.1,0.37,1,6,NW,ID0_audio1,0,5,sameera
1,NaN,NaN,6,11,NW,ID0_audio1,0,5,sameera
2,NaN,NaN,11,16,NW,ID0_audio1,0,5,sameera
3,NaN,NaN,16,21,NW,ID0_audio1,0,5,sameera
4,NaN,NaN,21,26,NW,ID0_audio1,0,5,sameera


checking the relative maximum of the dataset

In [25]:
files_df.drop(['start(mins-secs)','end(mins-secs)'],axis=1,inplace=True)
files_df = files_df.reset_index()
files_df.head()

,index,start,end,NW/BNC/Intro,filename,id,len,detection
0,0,1,6,NW,ID0_audio1,0,5,sameera
1,1,6,11,NW,ID0_audio1,0,5,sameera
2,2,11,16,NW,ID0_audio1,0,5,sameera
3,3,16,21,NW,ID0_audio1,0,5,sameera
4,4,21,26,NW,ID0_audio1,0,5,sameera


In [26]:
force_max_len=5

computing the length of the raw data of the slices

In [27]:
def compute_len(samp_rate=22050, time=force_max_len):
    '''Computes the supposed length of sliced data
        samp_rate = sampling rate. 
        time = length of time for the audio file. 
    '''
    comp_len = 0
    comp_len = samp_rate * time

    return comp_len

slice the wav file into subslices

In [28]:
def slice_data(start, end, raw_data,  sample_rate):
    max_ind = len(raw_data) 
    start_ind = min(int(start * sample_rate), max_ind)
    end_ind = min(int(end * sample_rate), max_ind)
    return raw_data[start_ind: end_ind]

In [29]:
# create output path
for d in ds:
    path = os.path.join('drive/MyDrive/EPITA/Signal Processing/Assi 1/subsliced_audios', d)
    os.makedirs(path, exist_ok=True)

In [30]:
i = 0 #iterator for file naming

for idx, row in files_df.iterrows():
    filename = row['filename']
    start = row['start']
    end = row['end']
    diag = row['detection']
     
    #check len and force to 'maxlen' secs if it is more than that
    if force_max_len < end - start:
        end = start + force_max_len
    
    aud_loc = audio_loc + '/' + filename + '.wav'
    
    if idx != 0:
        if files_df.iloc[idx-1]['filename'] == filename:
            i=i+1
        else:
            i=0
    n_filename = filename + '_' + str(i) + '.wav'
    path = 'drive/MyDrive/EPITA/Signal Processing/Assi 1/subsliced_audios/' + diag + '/' + n_filename
    
    print('processing ' + n_filename + '...')

    data, samplingrate = lb.load(aud_loc)
    sliced_data = slice_data(start=start, end=end, raw_data=data, sample_rate=samplingrate)
    
    #pad audio if < forced_max_len
    a_len = compute_len(samp_rate=samplingrate)
    padded_data = lb.util.pad_center(data = sliced_data, size = a_len)

    sf.write(file=path, data=padded_data, samplerate=samplingrate)


processing ID0_audio1_0.wav...
processing ID0_audio1_1.wav...
processing ID0_audio1_2.wav...
processing ID0_audio1_3.wav...
processing ID0_audio1_4.wav...
processing ID0_audio1_5.wav...
processing ID0_audio1_6.wav...
processing ID0_audio2_0.wav...
processing ID0_audio2_1.wav...
processing ID0_audio2_2.wav...
processing ID0_audio2_3.wav...
processing ID0_audio2_4.wav...
processing ID0_audio2_5.wav...
processing ID0_audio2_6.wav...
processing ID0_audio2_7.wav...
processing ID0_audio2_8.wav...
processing ID0_audio2_9.wav...
processing ID0_audio2_10.wav...
processing ID1_audio1_0.wav...
processing ID1_audio1_1.wav...
processing ID1_audio1_2.wav...
processing ID1_audio1_3.wav...
processing ID1_audio1_4.wav...
processing ID1_audio1_5.wav...
processing ID1_audio1_6.wav...
processing ID1_audio2_0.wav...
processing ID1_audio2_1.wav...
processing ID1_audio2_2.wav...
processing ID1_audio2_3.wav...
processing ID1_audio2_4.wav...
processing ID1_audio2_5.wav...
processing ID1_audio2_6.wav...
process